# Quafu Runtime Server

## 使用方式

### 1. 如何使用Quafu Runtime
创建RuntimeService

In [1]:
from clients.account import Account
from job.job import Job
from quafu_runtime_service import RuntimeService
# 获得用户与server交互的RuntimeService
# RuntimeService需要传入存有用户信息的Account
account = Account(apitoken="testapitoken")
service = RuntimeService(account)


### 2. 如何上传用户程序

In [ ]:
# 上传program，如果已经存在则失败
metadata = {"name": "long-run-task", "backend": "testbackend"}
id1 = service.upload_program(data='program/hello.py',metadata=metadata)
print(id1)

### 3. 如何获得已有用户程序
使用RuntimeService.programs以及Runtime.program接口

In [2]:
# 获得用户所有program的元数据
service.list_programs(refresh=True, detailed=False, limit=20)
# 获得program的所有数据
program = service.program(name="hello")
# 获得hello的program_id
print(program['program_id'])
# 获得hello的程序体
print(program['data'])
# program还有name,is_public,description,group,owner,backend属性

--------------------------------------------------
program_id:38c62c568b2643ee91f0625c214fca81
 -Name: for-while
 -Description: None
--------------------------------------------------
program_id:7a5f79b350214b279aa0f2cb6b85c811
 -Name: long-run-task
 -Description: None
--------------------------------------------------
program_id:7e55d4fb7ea44617adcd186049b458b6
 -Name: raise-exception
 -Description: None
--------------------------------------------------
program_id:d3ab487493ed4f048b8944c114dbe401
 -Name: hello
 -Description: None
d3ab487493ed4f048b8944c114dbe401
import numpy as np
from quafu import QuantumCircuit
from quafu import simulate
# used to test api
def prepare_circuits():
    q = QuantumCircuit(5)
    q.x(0)
    q.x(1)
    q.cnot(2, 1)
    q.ry(1, np.pi / 2)
    q.rx(2, np.pi)
    q.rz(3, 0.1)
    q.cz(2, 3)
    measures = [0, 1, 2, 3]
    cbits = [0, 1, 2, 3]
    q.measure(measures,  cbits=cbits)
    return q

def run(backend, userpub, params):
    """The entry point of th

### 4. 如何运行一个用户程序
使用RuntimeService.run接口

In [5]:
# 运行一个程序，可以通过指定name或program_id，返回Job实体
job1 = service.run(name='long-run-task',inputs={'TestParam1':'xxxxxx'})

### 5. 如何获得Job的中间结果
使用Job.interim_results接口

In [6]:
# 定义获得消息后的回调函数
def callback(job_id, message):
    print(message)
# 获得中间结果，该函数将以新的进程进行
job1.interim_results(callback=callback)

Start interim result streaming for job %s 57eaf485263c415299cd94791a1c2115
b'publish from job 10'
b'publish from job 9'
b'publish from job 8'
b'publish from job 7'
b'publish from job 6'
b'publish from job 5'
b'publish from job 4'
b'publish from job 3'
b'publish from job 2'
b'publish from job 1'
b'57eaf485263c415299cd94791a1c2115'
Interim result streaming finished


### 6. 如何获得Job的结果
使用Job.result接口

In [7]:
# 获得任务结果，无论运行完成，直接返回
result = job1.result(wait=False)
print(result)
# 获得任务结果，未完成则等待
result = job1.result(wait=True)
print(result)

{'finish_time': 'Wed, 10 May 2023 19:20:51 GMT', 'result': '{"result": {"TestParam1": "xxxxxx"}}', 'status': <JobStatus.DONE: 'Done'>}
{'result': '{"result": {"TestParam1": "xxxxxx"}}', 'finished_time': 'Wed, 10 May 2023 19:20:51 GMT', 'status': <JobStatus.DONE: 'Done'>}


### 7. 取消获得Job中间结果
使用Job.interim_result_cancel

In [13]:
# 取消获得中间结果
job1.interim_result_cancel()

Interim result streaming finished


### 8. 如何取消或删除一个Job
使用Job.cancel和Job.delete接口

In [13]:
# 取消Job
job1.cancel()
# 删除Job，但只会删除运行结束或取消了的任务
# job1.delete()

### 9. 如何获得任务的各项信息

In [14]:
# 获得job的日志
logs = job1.logs()
# 获得job的状态
status = job1.status()
# 获得错误信息
err_msg = job1.err_msg()
print(logs,status,err_msg)

Job status: JobStatus.DONE
Job c58c4d6693cd477ab1c4b6e9dc782c2a status: JobStatus.DONE
Hello World! {'TestParam1': 'xxxxxx'}
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
Sleep 10...
 JobStatus.DONE The job's status is: JobStatus.DONE
